In [1]:
import warnings

import numpy as np
import pandas as pd
import useful_rid_code as rid
from openpyxl import load_workbook

warnings.simplefilter("ignore")

# RID 

## Fill-in OSPAR reporting template (parameterised)

This notebook is "parameterised" for use with Papermill. The cell below has the tag `parameters`, which means the entire notebook can be called from `01_recalculate_ospar_1990-2016_main.ipynb`.

In [2]:
# This cell is tagged 'parameters' for use with Papermill
# https://papermill.readthedocs.io/en/latest/index.html
year = 1990
user = ""
pw = ""

In [3]:
# Parameters
user = "jes"
pw = "BeakabusNov21.."
year = 2012


## 1. Get summary data

The notebooks `summary_table_{year}.ipynb` calculate summary values for Table 3. Rather than reading these values from Word, it is easier to repeat the code to summarise the raw data again. This is the data that needs writing to the OSPAR template.

### 1.1. Monitored areas

In [4]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/loads_and_flows_all_sites_{year}.csv"
mon_df = pd.read_csv(in_csv)
del mon_df["new_rid_group"]

# Group by OSPAR region
mon_df1 = mon_df.groupby(["ospar_region", "old_rid_group"]).sum()

# Totals for Norway
mon_df2 = mon_df.groupby("old_rid_group").sum().reset_index()
mon_df2["ospar_region"] = "NORWAY"
mon_df2.set_index(["ospar_region", "old_rid_group"], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split("_")[1] != "Est"]
mon_df = mon_df[cols]
del mon_df["station_id"], mon_df["mean_q_1000m3/day"]

# Convert units
mon_df["Hg_kg"] = mon_df["Hg_kg"] / 1000.0  # kg to tonnes
mon_df["NH4-N_tonnes"] = mon_df["NH4-N_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["NO3-N_tonnes"] = mon_df["NO3-N_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["TOTN_tonnes"] = mon_df["TOTN_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["TOTP_tonnes"] = mon_df["TOTP_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["PO4-P_tonnes"] = mon_df["PO4-P_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["SPM_tonnes"] = mon_df["SPM_tonnes"] / 1000.0  # tonnes to ktonnes

# Units are correct, so remove
mon_df.columns = [i.split("_")[0] for i in mon_df.columns]

mon_df.round(0)

As   Cd    Cr    Cu   Hg  NH4-N  NO3-N  \
ospar_region        old_rid_group                                             
LOFOTEN-BARENTS SEA rid_108         1.0  0.0   1.0   5.0  0.0    0.0    0.0   
                    rid_11          0.0  0.0   1.0   2.0  0.0    0.0    0.0   
                    rid_36          2.0  0.0   3.0  16.0  0.0    0.0    1.0   
NORTH SEA           rid_108         1.0  0.0   1.0   8.0  0.0    0.0    3.0   
                    rid_11          0.0  0.0   0.0   2.0  0.0    0.0    0.0   
                    rid_36          2.0  0.0   2.0   9.0  0.0    0.0    3.0   
NORWEGIAN SEA2      rid_108         1.0  0.0   1.0   5.0  0.0    0.0    1.0   
                    rid_11          1.0  0.0   1.0  18.0  0.0    0.0    1.0   
                    rid_36          3.0  0.0  32.0  28.0  0.0    0.0    2.0   
SKAGERAK            rid_108         1.0  0.0   1.0   3.0  0.0    0.0    1.0   
                    rid_11         10.0  1.0  23.0  72.0  0.0    1.0   14.0   
                    rid_36          2.0  0.0   1.0   6.0  0.0    0.0    1.0   
NORWAY              rid_108         4.0  0.0   4.0  21.0  0.0    0.0    5.0   
                    rid_11         11.0  1.0  26.0  94.0  0.0    1.0   15.0   
                    rid_36          8.0  1.0  37.0  59.0  0.0    1.0    7.0   

                                     Ni  PO4-P    Pb    SPM      SiO2  \
ospar_region        old_rid_group                                       
LOFOTEN-BARENTS SEA rid_108         4.0    0.0   0.0    7.0   18560.0   
                    rid_11          1.0    0.0   0.0    5.0   14410.0   
                    rid_36         26.0    0.0   1.0   21.0   59866.0   
NORTH SEA           rid_108         6.0    0.0   3.0   19.0   22152.0   
                    rid_11          1.0    0.0   1.0    9.0    3926.0   
                    rid_36          3.0    0.0   5.0   30.0   24629.0   
NORWEGIAN SEA2      rid_108         3.0    0.0   1.0   18.0   16105.0   
                    rid_11          4.0    0.0   1.0   24.0   14495.0   
                    rid_36         23.0    0.0   3.0  200.0   60259.0   
SKAGERAK            rid_108         2.0    0.0   1.0   13.0    9308.0   
                    rid_11         34.0    1.0  20.0  553.0  190854.0   
                    rid_36          3.0    0.0   3.0   14.0   17804.0   
NORWAY              rid_108        15.0    0.0   4.0   57.0   66125.0   
                    rid_11         39.0    1.0  21.0  591.0  223685.0   
                    rid_36         55.0    0.0  12.0  265.0  162557.0   

                                        TOC  TOTN  TOTP     Zn   Ag  
ospar_region        old_rid_group                                    
LOFOTEN-BARENTS SEA rid_108         19798.0   1.0   0.0    5.0  0.0  
                    rid_11          11102.0   1.0   0.0    6.0  0.0  
                    rid_36          31517.0   4.0   0.0   27.0  0.0  
NORTH SEA           rid_108         23811.0   4.0   0.0   24.0  0.0  
                    rid_11           4556.0   1.0   0.0    6.0  0.0  
                    rid_36          41392.0   5.0   0.0   54.0  0.0  
NORWEGIAN SEA2      rid_108         21225.0   2.0   0.0    7.0  0.0  
                    rid_11          15296.0   2.0   0.0   34.0  0.0  
                    rid_36          87341.0   6.0   0.0   66.0  0.0  
SKAGERAK            rid_108         15320.0   2.0   0.0   18.0  0.0  
                    rid_11         227812.0  27.0   1.0  682.0  0.0  
                    rid_36          38843.0   3.0   0.0   54.0  0.0  
NORWAY              rid_108         80154.0   9.0   0.0   54.0  0.0  
                    rid_11         258766.0  31.0   1.0  728.0  0.0  
                    rid_36         199093.0  19.0   1.0  202.0  0.0

### 1.2. Unmonitored areas

In [5]:
# Read data
in_csv = f"../../../Results/Unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace("RENSEANLEGG", "sew") for i in umon_df.columns]
umon_df.columns = [i.replace("INDUSTRI", "ind") for i in umon_df.columns]
umon_df.columns = [i.replace("_tonn", "") for i in umon_df.columns]
umon_df.columns = [i.replace("AQUAKULTUR", "fish") for i in umon_df.columns]

# Add missing columns if necessary
cols = ["sew_Hg", "sew_S.P.M.", "ind_Hg"]
for col in cols:
    if col not in umon_df.columns:
        umon_df[col] = 0

# Convert Hg to kgs
umon_df["sew_Hg"] = umon_df["sew_Hg"] * 1000
umon_df["ind_Hg"] = umon_df["ind_Hg"] * 1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,371276.0,12069.0,927.0,2288.0,316.0,54300.0,9382.0,33799.0,680.0,556.0,...,2927.0,0.0,0.0,0.0,5.0,12.0,1.0,1.0,9.0,678.0
LOFOTEN-BARENTS SEA,91327.0,1169.0,155.0,104.0,10.0,12423.0,2149.0,4436.0,91.0,93.0,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,155.0
NORTH SEA,129809.0,3572.0,354.0,442.0,137.0,18761.0,3231.0,13783.0,217.0,212.0,...,107.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,233.0
NORWEGIAN SEA2,140918.0,2577.0,317.0,801.0,97.0,23091.0,3998.0,12676.0,287.0,190.0,...,2440.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,289.0
SKAGERAK,9222.0,4750.0,101.0,940.0,72.0,24.0,4.0,2904.0,85.0,61.0,...,363.0,0.0,0.0,0.0,4.0,10.0,1.0,0.0,7.0,0.0


### 1.3. Loads for 11 main rivers

In [6]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/loads_and_flows_all_sites_{year}.csv"
rid11_df = pd.read_csv(in_csv, index_col=0)
del rid11_df["new_rid_group"]

# Get data for RID11
rid11_df = rid11_df.query('old_rid_group == "rid_11"')

# Tidy
del rid11_df["station_code"], rid11_df["station_name"]
del rid11_df["old_rid_group"], rid11_df["ospar_region"]
del rid11_df["mean_q_1000m3/day"]

cols = [i for i in rid11_df.columns if i.split("_")[1] != "Est"]
rid11_df = rid11_df[cols]

# Convert units
rid11_df["Hg_kg"] = rid11_df["Hg_kg"] / 1000.0  # kg to tonnes
rid11_df["NH4-N_tonnes"] = rid11_df["NH4-N_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["NO3-N_tonnes"] = rid11_df["NO3-N_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["TOTN_tonnes"] = rid11_df["TOTN_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["TOTP_tonnes"] = rid11_df["TOTP_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["PO4-P_tonnes"] = rid11_df["PO4-P_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["SPM_tonnes"] = rid11_df["SPM_tonnes"] / 1000.0  # tonnes to ktonnes

# Tidy cols
rid11_df.columns = [i.split("_")[0] for i in rid11_df.columns]

rid11_df.head()

,As,Cd,Cr,Cu,Hg,NH4-N,NO3-N,Ni,PO4-P,Pb,SPM,SiO2,TOC,TOTN,TOTP,Zn,Ag
station_id,,,,,,,,,,,,,,,,,
29612,2.274190,0.174675,9.539226,13.994537,0.014646,0.102328,2.924437,6.381981,0.295978,6.796275,297.892664,36909.781540,44847.229388,5.562498,0.430921,39.222296,0.012051
29779,0.399488,0.011395,0.874526,2.200829,0.002095,0.012488,0.087869,0.925917,0.004955,0.183741,4.832701,14410.482888,11101.923802,0.670891,0.023744,6.139505,0.005642
29821,0.204676,0.020830,0.232686,1.496927,0.002128,0.024038,0.262609,0.919301,0.004244,0.260380,2.329048,3059.858709,3220.269062,0.562740,0.014984,4.325792,0.009391
29782,0.665240,0.016601,0.707329,3.335677,0.002255,0.011575,0.203292,1.409765,0.007999,0.581490,19.605203,6540.208292,7236.514056,0.668607,0.025589,3.152457,0.006732
36225,0.019556,0.002302,0.036925,0.197867,0.000013,0.007484,0.047967,0.050287,0.002622,0.047924,0.105756,334.073342,225.529236,0.076653,0.003069,0.587893,0.000489


## 2. Fill-in template

The template is usually sent each year by Csilla at NIBIO. However, it doesn't seem to change, so old versions can also be used e.g. here:

    ../../../Results/OSPAR/Blank_Template

In [7]:
# Copy of template to update
temp_path = rid.copy_ospar_template(year)

In [8]:
def update_spreadsheet_point_sources(xlsx, sheet, pars, src, df):
    """Update the OSPAR template for point source data.

    Args:
        xslx:  Str. Path to Excel template
        sheet: Str. Sheet name to update
        pars:  List. Parameter names in template to fill-in
        src:   Str. Type of input ('sew', 'ind', 'fish')
        df:    Dataframe. Values to fill-in

    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {
        "SPM": "S.P.M.",
        "TOC": "TOC",
        "PO4-P": "po4",
        "P-Total": "p",
        "NO3-N": "no3",
        "NH4-N": "nh4",
        "N-Total": "n",
        "As": "As",
        "Pb": "Pb",
        "Cd": "Cd",
        "Cu": "Cu",
        "Zn": "Zn",
        "Ni": "Ni",
        "Total Cr": "Cr",
        "Hg": "Hg",
    }

    # Map template names to df names
    names_dict = {
        "Norwegian Sea (NO)": "NORWEGIAN SEA2",
        "Barents Sea (NO)": "LOFOTEN-BARENTS SEA",
        "Skagerrak (NO)": "SKAGERAK",
        "North Sea (NO)": "NORTH SEA",
        "Norway Total": "NORWAY",
    }

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws["B12" : "B%s" % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws["E9":"AK9"][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        for par in pars:
            # Get value from df
            try:
                val = df.loc[names_dict[reg], "%s_%s" % (src, par_dict[par])]
            except KeyError:
                val = 0

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

### 2.1. Sheet 5a: Sewage effluents

Note these are the values from Table 3 for **unmonitored areas** and not the total sewage inputs for each of the OSPAR areas. This is the same as what Tore reported previously.

In [9]:
# Get Sewage data
cols = [i for i in umon_df.columns if i.split("_")[0] == "sew"]
sew_df = umon_df[cols].copy()

# Convert units
sew_df["sew_Hg"] = sew_df["sew_Hg"] / 1000.0  # kg to tonnes
sew_df["sew_nh4"] = sew_df["sew_nh4"] / 1000.0  # tonnes to ktonnes
sew_df["sew_no3"] = sew_df["sew_no3"] / 1000.0  # tonnes to ktonnes
sew_df["sew_n"] = sew_df["sew_n"] / 1000.0  # tonnes to ktonnes
sew_df["sew_po4"] = sew_df["sew_po4"] / 1000.0  # tonnes to ktonnes
sew_df["sew_p"] = sew_df["sew_p"] / 1000.0  # tonnes to ktonnes
sew_df["sew_S.P.M."] = sew_df["sew_S.P.M."] / 1000.0  # tonnes to ktonnes

sew_df

,sew_n,sew_p,sew_po4,sew_no3,sew_nh4,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg
ospar_region,,,,,,,,,,,,,,
NORWAY,12.068553,0.926710,0.556026,0.603428,9.051415,2.927498,0.192202,0.292089,0.012260,5.281358,11.506822,1.361885,0.595929,0.009357
LOFOTEN-BARENTS SEA,1.168683,0.154744,0.092847,0.058434,0.876512,0.017192,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NORTH SEA,3.572252,0.353643,0.212186,0.178613,2.679189,0.106849,0.012704,0.041705,0.001471,0.394240,1.127687,0.070294,0.043296,0.000581
NORWEGIAN SEA2,2.577358,0.316991,0.190195,0.128868,1.933019,2.440330,0.036716,0.055762,0.001315,0.956922,0.670505,0.208012,0.309558,0.001363
SKAGERAK,4.750259,0.101331,0.060799,0.237513,3.562694,0.363128,0.142782,0.194622,0.009474,3.930196,9.708630,1.083579,0.243075,0.007413


In [10]:
# Update sheet 5a
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
]

update_spreadsheet_point_sources(temp_path, "5a", pars, "sew", sew_df)

### 2.2. Sheet 5b: Industrial effluents

In [11]:
# Get industrial data
cols = [i for i in umon_df.columns if i.split("_")[0] == "ind"]
ind_df = umon_df[cols].copy()

# Convert units
ind_df["ind_Hg"] = ind_df["ind_Hg"] / 1000.0  # kg to tonnes
ind_df["ind_nh4"] = ind_df["ind_nh4"] / 1000.0  # tonnes to ktonnes
ind_df["ind_no3"] = ind_df["ind_no3"] / 1000.0  # tonnes to ktonnes
ind_df["ind_n"] = ind_df["ind_n"] / 1000.0  # tonnes to ktonnes
ind_df["ind_po4"] = ind_df["ind_po4"] / 1000.0  # tonnes to ktonnes
ind_df["ind_p"] = ind_df["ind_p"] / 1000.0  # tonnes to ktonnes
ind_df["ind_S.P.M."] = ind_df["ind_S.P.M."] / 1000.0  # tonnes to ktonnes

ind_df

,ind_n,ind_p,ind_po4,ind_no3,ind_nh4,ind_S.P.M.,ind_TOC,ind_As,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg
ospar_region,,,,,,,,,,,,,,,
NORWAY,2.287613,0.315847,0.189508,0.114381,1.715709,2637.250169,2414.9415,2.070241,4.379778,0.157806,5.717651,15.511241,8.956233,0.00716,0.007981
LOFOTEN-BARENTS SEA,0.103563,0.009896,0.005938,0.005178,0.077672,220.450233,250.9770,0.038838,3.134012,0.050000,0.199900,1.012000,1.720427,0.00000,0.000544
NORTH SEA,0.442320,0.136633,0.081980,0.022116,0.331740,5.977214,295.9877,1.836397,0.950194,0.068257,0.522992,8.820678,3.951579,0.00642,0.002469
NORWEGIAN SEA2,0.801468,0.097384,0.058431,0.040073,0.601101,2409.596151,135.1408,0.020085,0.067156,0.004355,0.093008,0.191682,0.214253,0.00074,0.000391
SKAGERAK,0.940261,0.071933,0.043160,0.047013,0.705196,1.226570,1732.8360,0.174922,0.228416,0.035194,4.901752,5.486881,3.069974,0.00000,0.004577


In [12]:
# Update sheet 5b
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

update_spreadsheet_point_sources(temp_path, "5b", pars, "ind", ind_df)

### 2.3. Sheet 5c: Aquaculture discharges

In [13]:
# Get fish data
cols = [i for i in umon_df.columns if i.split("_")[0] == "fish"]
fish_df = umon_df[cols].copy()

# Convert units
fish_df["fish_nh4"] = fish_df["fish_nh4"] / 1000.0  # tonnes to ktonnes
fish_df["fish_no3"] = fish_df["fish_no3"] / 1000.0  # tonnes to ktonnes
fish_df["fish_n"] = fish_df["fish_n"] / 1000.0  # tonnes to ktonnes
fish_df["fish_po4"] = fish_df["fish_po4"] / 1000.0  # tonnes to ktonnes
fish_df["fish_p"] = fish_df["fish_p"] / 1000.0  # tonnes to ktonnes

fish_df

,fish_n,fish_p,fish_po4,fish_no3,fish_nh4,fish_Cu
ospar_region,,,,,,
NORWAY,54.299722,9.381878,6.473496,5.972969,43.439778,678.300000
LOFOTEN-BARENTS SEA,12.423387,2.149079,1.482864,1.366573,9.938710,155.308133
NORTH SEA,18.761219,3.230786,2.229242,2.063734,15.008976,233.480196
NORWEGIAN SEA2,23.091216,3.997993,2.758615,2.540034,18.472973,289.221116
SKAGERAK,0.023899,0.004021,0.002774,0.002629,0.019119,0.290555


In [14]:
# Update sheet 5c
pars = ["NH4-N", "NO3-N", "P-Total", "PO4-P", "N-Total", "Cu"]

update_spreadsheet_point_sources(temp_path, "5c", pars, "fish", fish_df)

### 2.4. Sheet 5d: Other discharges

This sheet is left blank

### 2.5. Sheet 5e: Total direct discharges

The sum of sewage, industrial and fish-farm discharges.

In [15]:
# Combine sew, ind and fish, then aggregate
for df in [sew_df, ind_df, fish_df]:
    df.reset_index(inplace=True)
    df.columns = [i.split("_")[1] for i in df.columns]

td_df = pd.concat([sew_df, ind_df, fish_df], axis=0, sort=True)
td_df = td_df.groupby("region").sum()

td_df.columns = ["td_" + i for i in td_df.columns]

td_df

,td_As,td_Cd,td_Cr,td_Cu,td_Hg,td_Ni,td_Pb,td_S.P.M.,td_TOC,td_Zn,td_n,td_nh4,td_no3,td_p,td_po4
region,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,0.038838,0.050000,0.000000,155.508033,0.000544,1.720427,3.134012,220.467424,250.9770,1.012000,13.695633,10.892894,1.430185,2.313719,1.581649
NORTH SEA,1.849101,0.069728,0.049716,234.397428,0.003050,4.021873,0.991899,6.084063,295.9877,9.948365,22.775792,18.019905,2.264463,3.721062,2.523408
NORWAY,2.262443,0.170066,0.603089,689.299009,0.017338,10.318118,4.671867,2640.177667,2414.9415,27.018063,68.655887,54.206902,6.690778,10.624435,7.219030
NORWEGIAN SEA2,0.056801,0.005670,0.310298,290.271046,0.001753,0.422265,0.122918,2412.036481,135.1408,0.862187,26.470043,21.007093,2.708975,4.412369,3.007241
SKAGERAK,0.317704,0.044668,0.243075,9.122503,0.011990,4.153552,0.423038,1.589698,1732.8360,15.195511,5.714419,4.287009,0.287155,0.177285,0.106733


In [16]:
# Update sheet 5e
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

update_spreadsheet_point_sources(temp_path, "5e", pars, "td", td_df)

### 2.6. Sheet 6a: Monitored rivers

**Does "Inner Oslofjord" in the template correspond to "Alna"?** If so, I can fill-in one additional row in this table.

In [17]:
mon_df.reset_index(inplace=True)
tot_df = mon_df.groupby("ospar_region").sum()
trib_df = mon_df[mon_df["old_rid_group"] != "rid_11"].groupby("ospar_region").sum()

In [18]:
def update_spreadsheet_monitored_rivers(xlsx, sheet, pars, df_dict):
    """Update the OSPAR template for monitored rivers.

    Args:
        xslx:    Str. Path to Excel template
        sheet:   Str. Sheet name to update
        pars:    List. Parameter names in template to fill-in
        df_dict: Dict. {'tot':tot_df, 'trib':trib_df, 'main':rid11_df}
                 Values to fill-in

    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {
        "SPM": "SPM",
        "TOC": "TOC",
        "PO4-P": "PO4-P",
        "P-Total": "TOTP",
        "NO3-N": "NO3-N",
        "NH4-N": "NH4-N",
        "N-Total": "TOTN",
        "As": "As",
        "Pb": "Pb",
        "Cd": "Cd",
        "Cu": "Cu",
        "Zn": "Zn",
        "Ni": "Ni",
        "Total Cr": "Cr",
        "Hg": "Hg",
    }

    # Map template names to df names and rows
    names_dict = {
        "Norwegian Sea (NO)": ("NORWEGIAN SEA2", "tot"),
        "Barents Sea (NO)": ("LOFOTEN-BARENTS SEA", "tot"),
        "Skagerrak (NO)": ("SKAGERAK", "tot"),
        "North Sea (NO)": ("NORTH SEA", "tot"),
        "Norway Total": ("NORWAY", "tot"),
        "Tributary Rivers - Norwegian Sea": ("NORWEGIAN SEA2", "trib"),
        "Tributary Rivers - Barents Sea": ("LOFOTEN-BARENTS SEA", "trib"),
        "Tributary Rivers - Skagerak": ("SKAGERAK", "trib"),
        "Tributary Rivers - North Sea": ("NORTH SEA", "trib"),
        "Orkla": (29778, "main"),
        "Vefsna": (29782, "main"),
        "Alta": (29779, "main"),
        "Glomma": (29617, "main"),
        "Drammenselva": (29612, "main"),
        u"Numedalslågen": (29615, "main"),
        "Skienselva": (29613, "main"),
        "Otra": (29614, "main"),
        "Orreelva": (29783, "main"),
        "Vosso": (29821, "main"),
    }

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws["B12" : "B%s" % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws["E9":"AK9"][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        df_idx, df_name = names_dict[reg]

        # Get df
        df = df_dict[df_name]

        for par in pars:
            # Get value from df
            val = df.loc[df_idx, par_dict[par]]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

In [19]:
# Update sheet 6a
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]
df_dict = {"tot": tot_df, "main": rid11_df, "trib": trib_df}

update_spreadsheet_monitored_rivers(temp_path, "6a", pars, df_dict)

### 2.7. Sheet 6b: Unmonitored areas

In [20]:
# Get diff data
cols = [i for i in umon_df.columns if i.split("_")[0] == "diff"]
diff_df = umon_df[cols].copy()

# Convert units
diff_df["diff_nh4"] = diff_df["diff_nh4"] / 1000.0  # tonnes to ktonnes
diff_df["diff_no3"] = diff_df["diff_no3"] / 1000.0  # tonnes to ktonnes
diff_df["diff_n"] = diff_df["diff_n"] / 1000.0  # tonnes to ktonnes
diff_df["diff_po4"] = diff_df["diff_po4"] / 1000.0  # tonnes to ktonnes
diff_df["diff_p"] = diff_df["diff_p"] / 1000.0  # tonnes to ktonnes

diff_df

,diff_n,diff_p,diff_po4,diff_no3,diff_nh4
ospar_region,,,,,
NORWAY,33.799103,0.679671,0.167199,21.124440,1.858951
LOFOTEN-BARENTS SEA,4.436189,0.090929,0.022369,2.772618,0.243990
NORTH SEA,13.783227,0.217291,0.053454,8.614517,0.758077
NORWEGIAN SEA2,12.675749,0.286519,0.070484,7.922343,0.697166
SKAGERAK,2.903939,0.084932,0.020893,1.814962,0.159717


In [21]:
# Update sheet 6b
pars = ["NH4-N", "NO3-N", "P-Total", "PO4-P", "N-Total"]

update_spreadsheet_point_sources(temp_path, "6b", pars, "diff", diff_df)

### 2.8. Sheet 6c: Total inputs

**Note:** See e-mail from Csilla received 07/11/2017 at 13.43. This table should **not** include "point" discharges (`td_df`) - it's just the sum of tables 6a and 6b.

In [22]:
# Rename cols in diff_df
col_map = {
    "diff_n": "TOTN",
    "diff_p": "TOTP",
    "diff_po4": "PO4-P",
    "diff_no3": "NO3-N",
    "diff_nh4": "NH4-N",
}
diff_df.columns = [col_map[i] for i in diff_df.columns]

# Add to total_df
for col in diff_df.columns:
    tot_df[col] = tot_df[col] + diff_df[col]

# Update sheet 6c
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

df_dict = {"tot": tot_df, "main": rid11_df, "trib": trib_df}

update_spreadsheet_monitored_rivers(temp_path, "6c", pars, df_dict)

The following two cells are not longer used as I originally misunderstood what table 6c represents.

In [23]:
## Standardise col names
# diff_df.columns = [i.split('_')[1] for i in diff_df.columns]
# td_df.columns = [i.split('_')[1] for i in td_df.columns]
# td_df.index.name = 'ospar_region'
#
# col_dict = {'SPM':'S.P.M.',
#            'TOTN':'n',
#            'NH4-N':'nh4',
#            'NO3-N':'no3',
#            'TOTP':'p',
#            'PO4-P':'po4'}
# for col in col_dict.keys():
#    new_col = col_dict[col]
#    tot_df[new_col] = tot_df[col]
#    del tot_df[col]
#
## Reset index
# diff_df.reset_index(inplace=True)
# tot_df.reset_index(inplace=True)
# td_df.reset_index(inplace=True)
#
## Concat and aggregate
##tot_df = pd.concat([diff_df, td_df, tot_df], axis=0).groupby('ospar_region').sum() # See comment above
# tot_df = pd.concat([diff_df, tot_df], axis=0).groupby('ospar_region').sum()
#
## Rename cols
# tot_df.columns = ['tot_'+i for i in tot_df.columns]
#
# tot_df

In [24]:
## Update sheet 6c
# pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N',
#        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P',
#        'N-Total', 'Hg', 'Cu', 'TOC']
#
# update_spreadsheet_point_sources(temp_path, '6c', pars, 'tot', tot_df)

### 2.9. Sheet 7: Concentrations

In [25]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/concs_and_flows_rid_11_{year}.csv"
conc_df = pd.read_csv(in_csv, index_col=0, encoding="utf-8")

cols = [
    "Hg_ng/l",
    "NH4-N_µg/l N",
    "NO3-N_µg/l N",
    "TOTN_µg/l N",
    "TOTP_µg/l P",
    "PO4-P_µg/l P",
    "TOC_mg C/l",
]
for col in cols:
    if col not in conc_df.columns:
        par = col.split("_")[0]
        conc_df[col] = np.nan
        conc_df[f"{par}_flag"] = np.nan

# Convert units
conc_df["Hg_ng/l"] = conc_df["Hg_ng/l"] / 1000  # ng to ug
conc_df["NH4-N_µg/l N"] = conc_df["NH4-N_µg/l N"] / 1000  # ug to mg
conc_df["NO3-N_µg/l N"] = conc_df["NO3-N_µg/l N"] / 1000  # ug to mg
conc_df["TOTN_µg/l N"] = conc_df["TOTN_µg/l N"] / 1000  # ug to mg
conc_df["TOTP_µg/l P"] = conc_df["TOTP_µg/l P"] / 1000  # ug to mg
conc_df["PO4-P_µg/l P"] = conc_df["PO4-P_µg/l P"] / 1000  # ug to mg
conc_df["TOC_mg C/l"] = conc_df["TOC_mg C/l"] * 1000  # mg to ug

# Get flags
cols = [i for i in conc_df.columns if i.split("_")[1] == "flag"]
lod_df = conc_df[cols]
lod_df.columns = [i.split("_")[0] for i in lod_df.columns]

# Get vals
cols = [
    i
    for i in conc_df.columns
    if ((i.split("_")[0] in lod_df.columns) and (i.split("_")[1] != "flag"))
]
conc_df = conc_df[cols]
conc_df.columns = [i.split("_")[0] for i in conc_df.columns]

# Rename
col_dict = {
    "SPM": "S.P.M.",
    "TOTN": "n",
    "NH4-N": "nh4",
    "NO3-N": "no3",
    "TOTP": "p",
    "PO4-P": "po4",
}
for col in col_dict.keys():
    new_col = col_dict[col]
    lod_df[new_col] = lod_df[col]
    conc_df[new_col] = conc_df[col]
    del conc_df[col], lod_df[col]

# Map Excel headings to df cols
par_dict = {
    "SPM": "S.P.M.",
    "TOC": "TOC",
    "PO4-P": "po4",
    "P-Total": "p",
    "NO3-N": "no3",
    "NH4-N": "nh4",
    "N-Total": "n",
    "As": "As",
    "Pb": "Pb",
    "Cd": "Cd",
    "Cu": "Cu",
    "Zn": "Zn",
    "Ni": "Ni",
    "Total Cr": "Cr",
    "Hg": "Hg",
}

# Map names to stns
names_dict = {
    "Orkla": 29778,
    "Vefsna": 29782,
    "Alta": 29779,
    "Glomma": 29617,
    "Drammenselva": 29612,
    "Numedalslågen": 29615,
    "Skienselva": 29613,
    "Otra": 29614,
    "Orreelva": 29783,
    "Vosso": 29821,
}

# Open new file and get sheet
wb = load_workbook(filename=temp_path)
ws = wb["7"]

# Get row numbers
row_dict = {}
for item in ws["B12" : "B%s" % ws.max_row]:
    # Get cell properties
    cell = item[0]
    name = cell.value
    row = cell.row
    row_dict[name] = row

# Get col numbers
col_dict = {}
for cell in ws["E9":"AK9"][0]:
    # Get cell properties
    par = cell.value
    col = cell.column
    col_dict[par] = col

pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

# Update spreadsheet
for reg in names_dict.keys():
    for par in pars:
        if par_dict[par] not in conc_df.columns:
            conc_df[par_dict[par]] = np.nan
            lod_df[par_dict[par]] = np.nan

        # Get values from df
        # 1a. Lower average
        vals = conc_df.loc[names_dict[reg]].copy()[[par_dict[par]]].values
        lods = lod_df.loc[names_dict[reg]].copy()[[par_dict[par]]].fillna("0").values
        vals[(lods == "<")] = 0
        val = np.nanmean(vals)

        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 1b. Upper average
        val = conc_df.loc[names_dict[reg], par_dict[par]].mean()
        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 2. Min
        val = conc_df.loc[names_dict[reg], par_dict[par]].min()
        row = row_dict[reg] + 2
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 3. Max
        val = conc_df.loc[names_dict[reg], par_dict[par]].max()
        row = row_dict[reg] + 3
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 4. N
        n_samp = len(conc_df.loc[names_dict[reg]].copy()[[par_dict[par]]].dropna())
        if n_samp == 0:
            n_samp = np.nan
        row = row_dict[reg] + 5
        col = col_dict[par]
        ws.cell(row=row, column=col).value = n_samp

        # 5. N LOD
        n_lod = (~pd.isnull(lod_df.loc[names_dict[reg]].copy()[[par_dict[par]]])).sum()
        try:
            pct_lod = 100 * float(n_lod) / float(n_samp)
            if pct_lod < 30:
                val = "Yes"
            else:
                val = "No"
        except ZeroDivisionError:
            val = np.nan

        row = row_dict[reg] + 4
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 6. Std
        val = conc_df.loc[names_dict[reg], par_dict[par]].std()
        row = row_dict[reg] + 7
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

# Save
wb.save(temp_path)

### 2.10. Sheet 9: Discharge

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/recalculate_ospar_flows.ipynb) handles the OSPAR flow data. Run this, and then copy the results over to Sheet 9. 